## Info

This is a testing notebook for the MM Estimator in R.

Its stems from there being issues in what version of R is installed relative to Python, Anaconda and rpy2.

<font color=red size=4> **Its important to test multiple environments (kernels) here!** </font>
- fnb
- test


It uses the Boston Data Set from sklearn as test data.

<font color='purple' size=4>**Tests both MM Estimators in the Function.**</font>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sma

import os
import gc

pd.set_option('max_columns', 150)
pd.set_option('max_rows', 100)

### Import Data and Elastic Net

In [3]:
import sklearn.datasets

In [4]:
boston = sklearn.datasets.load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=['target'])

In [5]:
X.head()

CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  
0     15.3  396.90   4.98  
1     17.8  396.90   9.14  
2     17.8  392.83   4.03  
3     18.7  394.63   2.94  
4     18.7  396.90   5.33

In [6]:
y.head()

target
0    24.0
1    21.6
2    34.7
3    33.4
4    36.2

### Create a DataFrame

In [7]:
## create a dataframe like you do for SAS OUT and standardize (the model didnt like unstandardized data)
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
xs = pd.DataFrame(ss.fit_transform(X), columns=X.columns)
ys = pd.Series(ss.fit_transform(y).flatten()); ys.name='target'

rdf = pd.concat([y,xs], axis=1)
rdf.head()

target      CRIM        ZN     INDUS      CHAS       NOX        RM  \
0    24.0 -0.419782  0.284830 -1.287909 -0.272599 -0.144217  0.413672   
1    21.6 -0.417339 -0.487722 -0.593381 -0.272599 -0.740262  0.194274   
2    34.7 -0.417342 -0.487722 -0.593381 -0.272599 -0.740262  1.282714   
3    33.4 -0.416750 -0.487722 -1.306878 -0.272599 -0.835284  1.016303   
4    36.2 -0.412482 -0.487722 -1.306878 -0.272599 -0.835284  1.228577   

        AGE       DIS       RAD       TAX   PTRATIO         B     LSTAT  
0 -0.120013  0.140214 -0.982843 -0.666608 -1.459000  0.441052 -1.075562  
1  0.367166  0.557160 -0.867883 -0.987329 -0.303094  0.441052 -0.492439  
2 -0.265812  0.557160 -0.867883 -0.987329 -0.303094  0.396427 -1.208727  
3 -0.809889  1.077737 -0.752922 -1.106115  0.113032  0.416163 -1.361517  
4 -0.511180  1.077737 -0.752922 -1.106115  0.113032  0.441052 -1.026501

### Set up R Home

In [8]:
# help(RRobustMMEstimator.RMM_MASS)

In [9]:
import os
##  you must then set your R HOME before importing the module - it should be in a similar location as below

os.environ['R_HOME'] = 'C:/Users/schillw/Anaconda3/envs/fnb/Lib/R'


from fnbbase import RRobustMMEstimator

In [10]:
load_ext rpy2.ipython

C:\Users\schillw\Anaconda3\envs\fnb\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  "The symbol '%s' is not in this R namespace/package." % name


In [16]:
yy = ys.values[:, None]
xx = xs.values#[:, None]

In [17]:
xx

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

In [18]:
%R library(robustbase)
%Rpush yy xx
%R mod <- lmrob(yy ~ xx);
%R params <- mod$coefficients;
%Rpull params

In [19]:
%R print(mod)


Call:
lmrob(formula = yy ~ xx)
 \--> method = "MM"
Coefficients:
(Intercept)          xx1          xx2          xx3          xx4          xx5  
   -0.09067     -0.11725      0.06947     -0.01046      0.03409     -0.07721  
        xx6          xx7          xx8          xx9         xx10         xx11  
    0.47176     -0.12760     -0.21856      0.14360     -0.20394     -0.16395  
       xx12         xx13  
    0.12421     -0.16921  



<rpy2.robjects.vectors.ListVector object at 0x0000020B2E90FC88> [RTYPES.VECSXP]
R classes: ('lmrob',)
[FloatSexp..., FloatSexp..., FloatSexp..., FloatSexp..., ..., LangSexpV..., IntSexpVe..., ListSexpV..., FloatSexp...]
  coefficients: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x0000020B2639ED48> [RTYPES.REALSXP]
R classes: ('numeric',)
[-0.090667, -0.117255, 0.069471, -0.010460, ..., -0.203940, -0.163955, 0.124212, -0.169207]
  scale: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x0000020B2639E8C8> [RTYPES.REALSXP]
R classes: ('numeric',)
[0.311914]
  residuals: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x0000020B2EE79248> [RTYPES.REALSXP]
R classes: ('numeric',)
[-0.484586, -0.267535, 0.448331, 0.461333, ..., -0.090212, -0.335322, -0.379215, -0.963841]
  loss: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x0000020B2EE79048> [RTYPES.REALSXP]
R classes: ('numeric',)
[0.000000]
...
  iter: <class 'rpy2.robjects.Formula'>
  <rpy2.robjects.Formula object at 0x0000020B2639C948> [RTYPES.LANGSXP]
R classes: ('terms', 'formula')
  fitted.values: <class 'rpy2.robjects.vectors.IntVector'>
  <rpy2.robjects.vectors.IntVector object at 0x0000020B2639C4C8> [RTYPES.INTSXP]
R classes: ('integer',)
[0, 1, 1, 1, ..., 1, 1, 1, 1]
<rpy2.robjects.vectors.ListVector object at 0x0000020B2E90FC88> [RTYPES.VECSXP]
R classes: ('lmrob',)
[FloatSexp..., FloatSexp..., FloatSexp..., FloatSexp..., ..., LangSexpV..., IntSexpVe..., ListSexpV..., FloatSexp...]
  control: <class 'rpy2.robjects.vectors.FloatMatrix'>
  <rpy2.robjects.vectors.FloatMatrix object at 0x0000020B2639C588> [RTYPES.REALSXP]
R classes: ('matrix',)
[1.000000, 1.000000, 1.000000, 1.000000, ..., -0.500850, -0.983048, -0.865302, -0.669058]

**Interesting that the above worked...**

In [22]:
%R print(mod['coefficients'])

$coefficients
(Intercept)         xx1         xx2         xx3         xx4         xx5 
-0.09066658 -0.11725481  0.06947070 -0.01045974  0.03408572 -0.07720879 
        xx6         xx7         xx8         xx9        xx10        xx11 
 0.47175783 -0.12759859 -0.21855698  0.14360478 -0.20394023 -0.16395453 
       xx12        xx13 
 0.12421156 -0.16920714 



<rpy2.robjects.vectors.ListVector object at 0x0000020B2CDDD588> [RTYPES.VECSXP]
R classes: ('list',)
[FloatSexpVector]
  coefficients: <class 'rpy2.rinterface.FloatSexpVector'>
  <rpy2.rinterface.FloatSexpVector object at 0x0000020B2E913048> [RTYPES.REALSXP]

In [35]:
len([*params]), len(xs.columns)

(14, 13)

In [41]:
idx = ['intercept'] + list(xs.columns)
# print(idx)
pd.DataFrame([*params], index=idx)

0
intercept -0.090667
CRIM      -0.117255
ZN         0.069471
INDUS     -0.010460
CHAS       0.034086
NOX       -0.077209
RM         0.471758
AGE       -0.127599
DIS       -0.218557
RAD        0.143605
TAX       -0.203940
PTRATIO   -0.163955
B          0.124212
LSTAT     -0.169207

In [10]:
huber = RRobustMMEstimator.RMM_MASS(ys, xs, k2=1.345, psi_func='Huber', maxiter=1000)

In [12]:
huber_fit = huber.fit()

In [13]:
huber.print_summary()

               Model:          R Robust Estimator - MASS
                Date:                   2021-07-15 16:51
Estimator / Function:                         Huber / MM
                 Chi:                              1.345
                 Psi: function (u, c = 4.685, deriv = 0)
    No. Observations:                                506
            Df Model:                                 14
        Df Residuals:                                492
           Converged:                              False
          Scale Est.:                            0.31281
      Adj. R-squared:                 0.6844738898485905
              Estimate Std. Error    t value   Pr(>|t|)
-------------------------------------------------------
(Intercept)   -0.09043    0.01624   -5.56802        0.0
CRIM          -0.11697    0.02174   -5.37986        0.0
ZN             0.06953    0.02462    2.82373     0.0056
INDUS         -0.01044    0.03245   -0.32166     0.7437
CHAS           0.03411    0.01683    

In [15]:
tukey = RRobustMMEstimator.RMM_LMROB(ys, xs, s_breakdown=1.54764, m_breakdown=4.685061, psi_func='bisquare', maxiter=1000)

In [16]:
tukey_fit = tukey.fit()

In [17]:
tukey.print_summary()

               Model: R Robust Estimator - LMROB
                Date:           2021-07-15 16:52
Estimator / Function:              MM / bisquare
          Chi / Psi :         1.54764 / 4.685061
    No. Observations:                        506
            Df Model:                         14
        Df Residuals:                        492
           Converged:                       True
               Iters:                         49
          Scale Est.:                    0.31191
      Adj. R-squared:                    0.83694
             Estimate Std. Error    t value   Pr(>|t|)
------------------------------------------------------
Intercept    -0.09067    0.02301   -3.94015      9e-05
CRIM         -0.11725    0.04321   -2.71365    0.00689
ZN            0.06947    0.03668    1.89405     0.0588
INDUS        -0.01046     0.0255   -0.41016    0.68187
CHAS          0.03409     0.0165    2.06524    0.03942
NOX          -0.07721    0.04211   -1.83343    0.06734
RM            0.47176